## Software installation

This lab relies on a couple PyPI packages. If you don't have them installed, run the following cell:

In [ ]:
!pip install xgboost==1.7 scikit-learn pandas cleanlab

In [90]:
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np



# read in already preprocessed data
df = pd.read_csv("df.csv", index_col=0)
data = pd.read_csv('X.csv', index_col=0)
labels = pd.read_csv('Y.csv', index_col=0)
labels = labels['noisy_letter_grade']


# XGBoost(experimental) supports categorical data.
# Here we use default hyperparameters for simplicity.
# Get out-of-sample predicted probabilities and check model accuracy.
model = XGBClassifier(tree_method="hist", enable_categorical=True)
# pred_probs should be a Nx5 matrix of out-of-sample predicted probabilities, with N = len(data)

pred_probs = cross_val_predict(X=data, y = labels, estimator=model, cv=5, method='predict_proba')


# Excercise 0: Look at data and model outputs

In [91]:
### YOUR CODE HERE ####


## Checking model accuracy on original data

Now that we have out-of-sample predicted probabilities, we can also check the model's (cross-val) accuracy on the original (noisy) data, so we'll have a baseline to compare our final results.

In [92]:
preds = np.argmax(pred_probs, axis=1)
acc_original = accuracy_score(preds, labels)
print(f"Accuracy with original data: {round(acc_original*100,1)}%")
# should be: Accuracy with original data: 65.8%

Accuracy with original data: 65.8%


# Finding label issues automatically

We count label issues using confident learning. First, we need to compute class thresholds for the different classes.

# Exercise 1: computing class thresholds

Implement the Confident Learning algorithm for computing class thresholds for the 5 classes.

The class threshold for each class is the model's expected (average) self-confidence for each class. In other words, to compute the threshold for a particular class, you can average the predicted probability for that class, for all datapoints that are labeled with that particular class.

In [112]:
import numpy as np

def compute_class_thresholds(pred_probs: np.ndarray, labels: np.ndarray) -> np.ndarray:
    #### YOUR CODE HERE ####
    pass

In [ ]:
# should be a numpy array of length 5
thresholds = compute_class_thresholds(pred_probs, labels.to_numpy())
print(thresholds)
# results should be:
# [0.71534363 0.63193113 0.67064151 0.46515035 0.44225472]

# Exercise 2: constructing the confident joint

Next, we compute the confident joint, a matrix that counts the number of label errors for each noisy label $\tilde{y}$ and true label $y^*$.

The confident joint C is a K x K matrix (with K = 5 for this dataset), where `C[i][j]` is an estimate of the count of the number of data points with noisy label `i` and true label `j`. From lecture, recall that we put a data point in bin `(i, j)` if its given label is `i`, and its predicted probability for class `j` is above the threshold for class `j` (`thresholds[j]`). Each data point should only go in a single bin; if a data point's predicted probability is above the class threshold for multiple classes, it goes in the bin for which it has the highest predicted probability.

In [111]:
def compute_confident_joint(pred_probs: np.ndarray, labels: np.ndarray, thresholds: np.ndarray) -> np.ndarray:
    # YOUR CODE HERE
    pass

In [97]:
C = compute_confident_joint(pred_probs, labels.to_numpy(), thresholds)

# Exercise 3: count the number of label issues

Now that we have the confident joint C, we can count the estimated number of label issues in our dataset. Recall that this is the sum of the off-diagonal entries (the cases where we estimate that a label has been flipped).

In [113]:
num_label_issues = None # YOUR CODE HERE

In [ ]:
print('Estimated noise rate: {:.1f}%'.format(100*num_label_issues / pred_probs.shape[0]))
# should be: Estimated noise rate: 25.3%

# Exercise 4: filter out label issues

In this lab, our approach to identifying issues is to rank the data points by a score ("self-confidence", the model's predicted probability for a data point's given label) and then take the top `num_label_issues` of those.

First, we want to compute the model's _self-confidence_ for each data point. For a data point `i`, that is `pred_probs[i, labels[i]]`.

In [101]:
# this should be a numpy array of length 941 of probabilities
self_confidences = None # YOUR CODE HERE
# this should be a numpy array of length 941 of probabilities
self_confidences = np.array([pred_probs[i, l] for i, l in enumerate(labels)])

Next, we rank the _indices_ of the data points by the self-confidence.

In [ ]:
# this should be a numpy array of length 941 of integer indices
ranked_indices = None # YOUR CODE HERE
df.iloc[ranked_indices[:5]]

Finally, let's compute the indices of label issues as the top `num_label_issues` items in the `ranked_indices`.

In [115]:
issue_idx = None # YOUR CODE HERE

Let's look at a couple of the highest-ranked data points (most likely to be label issues):

In [ ]:
df.iloc[ranked_indices[:5]]

# How'd We Do?

Let's go a step further and see how we did at automatically identifying which data points are mislabeled. If we take the intersection of the labels errors identified by Confident Learning and the true label errors, we see that our approach was able to identify 75% of the label errors correctly (based on predictions from a model that is only 67% accurate). 

In [ ]:
# Computing percentage of true errors identified. 
true_error_idx = df[df.letter_grade != df.noisy_letter_grade].index.values
cl_acc = len(set(true_error_idx).intersection(set(issue_idx)))/len(true_error_idx)
print(f"Percentage of errors found: {round(cl_acc*100,1)}%")
# should be: Percentage of errors found: 77.7%

# Train a More Robust Model

Now that we have the indices of potential label errors within our data, let's remove them from our data, retrain our model, and see what improvement we can gain.

Keep in mind that our baseline model from above, trained on the original data using the `noisy_letter_grade` as the prediction label, achieved a cross-validation accuracy of 67%.

Let's use a very simple method to handle these label errors and just drop them entirely from the data and retrain our exact same `XGBClassifier`. In a real-world application, a better approach might be to have humans review the issues and _correct_ the labels rather than dropping the data points.

In [ ]:
# Remove the label errors found by Confident Learning
data = df.drop(issue_idx)
clean_labels = data['noisy_letter_grade']
data = data.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)

# Train a more robust classifier with less erroneous data
model = XGBClassifier(tree_method="hist", enable_categorical=True)
clean_pred_probs = cross_val_predict(model, data, clean_labels, method='predict_proba')
clean_preds = np.argmax(clean_pred_probs, axis=1)

acc_clean = accuracy_score(clean_preds, clean_labels)
print(f"Accuracy with original data: {round(acc_original*100, 1)}%")
print(f"Accuracy with errors found by Confident Learning removed: {round(acc_clean*100, 1)}%")

# Compute reduction in error.
err = ((1-acc_original)-(1-acc_clean))/(1-acc_original)
print(f"Reduction in error: {round(err*100,1)}%")
# should be:
# Accuracy with original data: 65.8%
# Accuracy with errors found by Confident Learning removed: 90.3%
# Reduction in error: 71.7%

After removing the suspected label issues, our model's new cross-validation accuracy is now 90%, which means we **reduced the error-rate of the model by 70%** (the original model had 67% accuracy). 

**Note: throughout this entire process we never changed any code related to model architecture/hyperparameters, training, or data preprocessing!  This improvement is strictly coming from increasing the quality of our data which leaves additional room for additional optimizations on the modeling side.**

# Conclusion

For the student grades dataset, we found that simply dropping identified label errors and retraining the model resulted in a 70% reduction in prediction error on our classification problem (with accuracy improving from 67% to 90%).

An implementation of the Confident Learning algorithm (and much more) is available in the [cleanlab](https://github.com/cleanlab/cleanlab) library on GitHub. This is how today's lab assignment can be done in a single line of code with Cleanlab:

In [109]:
import cleanlab

cl_issue_idx = cleanlab.filter.find_label_issues(labels, pred_probs, return_indices_ranked_by='self_confidence')

In [ ]:
df.iloc[cl_issue_idx[:5]]

In [ ]:
# Remove the label errors found by Confident Learning
data = df.drop(cl_issue_idx)
clean_labels = data['noisy_letter_grade']
data = data.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)

# Train a more robust classifier with less erroneous data
model = XGBClassifier(tree_method="hist", enable_categorical=True)
clean_pred_probs = cross_val_predict(model, data, clean_labels, method='predict_proba')
clean_preds = np.argmax(clean_pred_probs, axis=1)

acc_clean = accuracy_score(clean_preds, clean_labels)
print(f"Accuracy with original data: {round(acc_original*100, 1)}%")
print(f"Accuracy with errors found by Confident Learning removed: {round(acc_clean*100, 1)}%")

# Compute reduction in error.
err = ((1-acc_original)-(1-acc_clean))/(1-acc_original)
print(f"Reduction in error: {round(err*100,1)}%")

_Advanced topic_: you might notice that the above `cl_issue_idx` differs in length (by a little bit) from our `issue_idx`.Ssee equation 3 and the subsequent explanation in the [paper](https://jair.org/index.php/jair/article/view/12125).